In [2]:
import pandas as pd
import numpy as np

### Import the Tracking dataset

In [3]:
tracking_data = pd.read_csv("../data/raw/PlayerTrackData.csv")

In [4]:
tracking_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 76366748 entries, 0 to 76366747
Data columns (total 9 columns):
 #   Column   Dtype  
---  ------   -----  
 0   PlayKey  object 
 1   time     float64
 2   event    object 
 3   x        float64
 4   y        float64
 5   dir      float64
 6   dis      float64
 7   o        float64
 8   s        float64
dtypes: float64(7), object(2)
memory usage: 5.1+ GB


Sort dataset by PlayKey and Time to get a timeline of each player's actions.

In [6]:
tracking_sorted = tracking_data.sort_values(by=['PlayKey', 'time'])

In [7]:
tracking_aggregated = tracking_sorted.copy()

Calculate the acceleration of each player

In [ ]:
tracking_aggregated['dt'] = tracking_aggregated.groupby('PlayKey')['time'].diff()
tracking_aggregated['ds'] = tracking_aggregated.groupby('PlayKey')['s'].diff()
tracking_aggregated['acceleration'] = tracking_aggregated['ds'] / tracking_aggregated['dt']

In [9]:
tracking_aggregated.head(10)

,PlayKey,time,event,x,y,dir,dis,o,s,dt,ds,acceleration
0,26624-1-1,0.0,huddle_start_offense,87.46,28.93,288.24,0.01,262.33,0.13,NaN,NaN,NaN
1,26624-1-1,0.1,NaN,87.45,28.92,283.91,0.01,261.69,0.12,0.1,-0.01,-0.1
2,26624-1-1,0.2,NaN,87.44,28.92,280.40,0.01,261.17,0.12,0.1,0.00,0.0
3,26624-1-1,0.3,NaN,87.44,28.92,278.79,0.01,260.66,0.10,0.1,-0.02,-0.2
4,26624-1-1,0.4,NaN,87.44,28.92,275.44,0.01,260.27,0.09,0.1,-0.01,-0.1
5,26624-1-1,0.5,NaN,87.45,28.92,270.06,0.01,260.08,0.07,0.1,-0.02,-0.2
6,26624-1-1,0.6,NaN,87.46,28.92,265.05,0.01,260.05,0.05,0.1,-0.02,-0.2
7,26624-1-1,0.7,NaN,87.46,28.92,255.75,0.00,260.28,0.02,0.1,-0.03,-0.3
8,26624-1-1,0.8,NaN,87.46,28.92,244.56,0.00,260.72,0.01,0.1,-0.01,-0.1
9,26624-1-1,0.9,NaN,87.46,28.92,220.57,0.00,261.26,0.01,0.1,0.00,0.0


Create an aggregation dictionary for the dataset

In [ ]:
aggregation_dictionary = {
  's': ['mean', 'max'],
  'dis': ['sum'],
  'time': ['max'],
  'acceleration': ['mean', 'max'],
}

tracking_aggregated = tracking_aggregated.groupby(['PlayKey']).agg(aggregation_dictionary)


KeyError: "Column(s) ['acceleration', 'dis', 's', 'time'] do not exist"

In [13]:
tracking_aggregated.head(10)

s          dis  time acceleration     
                mean   max    sum   max         mean  max
PlayKey                                                  
26624-1-1   0.535753  2.94  16.83  29.8    -0.000336  2.7
26624-1-10  0.576304  3.18  17.07  27.5    -0.017455  2.5
26624-1-11  0.795772  2.94  31.20  36.8     0.064946  2.0
26624-1-12  0.349692  1.83  14.70  35.6     0.005056  2.6
26624-1-13  0.427109  1.49  13.26  25.5     0.005490  2.7
26624-1-14  0.617982  2.06  21.82  32.6     0.010123  2.4
26624-1-15  0.573657  1.68  22.70  36.0     0.019444  2.5
26624-1-16  0.768885  3.12  24.61  29.5     0.012203  2.5
26624-1-17  1.890593  4.55  95.49  48.8     0.062295  2.5
26624-1-18  0.869355  1.43  13.91  15.4     0.064286  2.0

Flatten the dataset 

In [14]:
tracking_aggregated.columns = [
    f"{col[0]}_{col[1]}" if col[1] not in ('<lambda>', '') else col[0]
    for col in tracking_aggregated.columns
]
tracking_aggregated.head(10)

,s_mean,s_max,dis_sum,time_max,acceleration_mean,acceleration_max
PlayKey,,,,,,
26624-1-1,0.535753,2.94,16.83,29.8,-0.000336,2.7
26624-1-10,0.576304,3.18,17.07,27.5,-0.017455,2.5
26624-1-11,0.795772,2.94,31.20,36.8,0.064946,2.0
26624-1-12,0.349692,1.83,14.70,35.6,0.005056,2.6
26624-1-13,0.427109,1.49,13.26,25.5,0.005490,2.7
26624-1-14,0.617982,2.06,21.82,32.6,0.010123,2.4
26624-1-15,0.573657,1.68,22.70,36.0,0.019444,2.5
26624-1-16,0.768885,3.12,24.61,29.5,0.012203,2.5
26624-1-17,1.890593,4.55,95.49,48.8,0.062295,2.5


In [16]:
tracking_aggregated.to_csv("../data/processed/tracking_aggregated.csv")